In [1]:
from numba import jit, cuda
import pandas as pd
import urllib.request
import os
from PIL import Image
import numpy
import tensorflow as tf
import concurrent.futures

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [27]:
qtd_train = 155
qtd_total = 200
path_data = "Planilhas/Coragyps-atratus.csv"
especie = 'Coragyps-atratus'

In [4]:
def download_and_process_image(url, path, size=(128, 128)):
    try:
        # Download da imagem
        urllib.request.urlretrieve(url, path)
        # Processamento da imagem
        image = Image.open(path).convert('RGB')
        image = image.resize(size)
        image.save(path, 'JPEG')
        return True
    except Exception as e:
        print(f"Erro ao processar {url}: {e}")
        return False

def salvaImagens(path_data, especie):
    data = pd.read_csv(path_data)
    data.dropna(inplace=True)
    train = data[:qtd_train]
    test = data[qtd_train:qtd_total]

    os.makedirs(f'./train/{especie}', exist_ok=True)
    os.makedirs(f'./test/{especie}', exist_ok=True)
    
    def process_images(dataset, folder):
        futures = []
        with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
            for i, url in enumerate(dataset['image_url']):
                path = f'{folder}/{especie}/imagem_{i}.jpg'
                futures.append(executor.submit(download_and_process_image, url, path))
            
            for future in concurrent.futures.as_completed(futures):
                if not future.result():
                    print("Um download ou processamento falhou.")
    
    process_images(train, './train')
    process_images(test, './test')

In [29]:
salvaImagens(path_data, especie)